![](image.jpg)


Dive into the heart of data science with a project that combines healthcare insights and predictive analytics. As a Data Scientist at a top Health Insurance company, you have the opportunity to predict customer healthcare costs using the power of machine learning. Your insights will help tailor services and guide customers in planning their healthcare expenses more effectively.

## Dataset Summary

Meet your primary tool: the `insurance.csv` dataset. Packed with information on health insurance customers, this dataset is your key to unlocking patterns in healthcare costs. Here's what you need to know about the data you'll be working with:

## insurance.csv
| Column    | Data Type | Description                                                      |
|-----------|-----------|------------------------------------------------------------------|
| `age`       | int       | Age of the primary beneficiary.                                  |
| `sex`       | object    | Gender of the insurance contractor (male or female).             |
| `bmi`       | float     | Body mass index, a key indicator of body fat based on height and weight. |
| `children`  | int       | Number of dependents covered by the insurance plan.              |
| `smoker`    | object    | Indicates whether the beneficiary smokes (yes or no).            |
| `region`    | object    | The beneficiary's residential area in the US, divided into four regions. |
| `charges`   | float     | Individual medical costs billed by health insurance.             |



A bit of data cleaning is key to ensure the dataset is ready for modeling. Once your model is built using the `insurance.csv` dataset, the next step is to apply it to the `validation_dataset.csv`. This new dataset, similar to your training data minus the `charges` column, tests your model's accuracy and real-world utility by predicting costs for new customers.

## Let's Get Started!

This project is your playground for applying data science in a meaningful way, offering insights that have real-world applications. Ready to explore the data and uncover insights that could revolutionize healthcare planning? Let's begin this exciting journey!

In [220]:
# Re-run this cell
# Import required libraries
import pandas as pd
import numpy as np
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder 
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LinearRegression

# Loading the insurance dataset
insurance_data_path = 'insurance.csv'
insurance = pd.read_csv(insurance_data_path)
insurance.head()

,age,sex,bmi,children,smoker,region,charges
0,19.0,female,27.900,0.0,yes,southwest,16884.924
1,18.0,male,33.770,1.0,no,Southeast,1725.5523
2,28.0,male,33.000,3.0,no,southeast,$4449.462
3,33.0,male,22.705,0.0,no,northwest,$21984.47061
4,32.0,male,28.880,0.0,no,northwest,$3866.8552


In [221]:
insurance.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1338 entries, 0 to 1337
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   age       1272 non-null   float64
 1   sex       1272 non-null   object 
 2   bmi       1272 non-null   float64
 3   children  1272 non-null   float64
 4   smoker    1272 non-null   object 
 5   region    1272 non-null   object 
 6   charges   1284 non-null   object 
dtypes: float64(3), object(4)
memory usage: 73.3+ KB


In [222]:
insurance_cat_df = insurance.drop('charges', axis=1).select_dtypes(include=['object'])

insurance_cat_df.apply(lambda x: x.value_counts()).T.stack()

sex     F              62.0
        M              64.0
        female        503.0
        male          517.0
        man            64.0
        woman          62.0
smoker  no           1013.0
        yes           259.0
region  Northeast     155.0
        Northwest     161.0
        Southeast     172.0
        Southwest     148.0
        northeast     153.0
        northwest     149.0
        southeast     170.0
        southwest     164.0
dtype: float64

In [223]:
insurance['sex'] = insurance['sex'].replace({'M':'male', 'man': 'male', 'F':'female', 'woman':'female'})
insurance['region'] = insurance['region'].str.lower()
insurance['age'] = insurance['age'].abs()
insurance['charges'] = insurance['charges'].str.replace('$','').astype('float64').round(2)

insurance.head()

,age,sex,bmi,children,smoker,region,charges
0,19.0,female,27.900,0.0,yes,southwest,16884.92
1,18.0,male,33.770,1.0,no,southeast,1725.55
2,28.0,male,33.000,3.0,no,southeast,4449.46
3,33.0,male,22.705,0.0,no,northwest,21984.47
4,32.0,male,28.880,0.0,no,northwest,3866.86


In [224]:
X = insurance.drop('charges', axis=1)
y = insurance['charges'].fillna(insurance['charges'].mean())

X_cat_features = insurance.drop('charges', axis=1).select_dtypes(include=['object']).columns.tolist()
X_num_features = insurance.drop('charges', axis=1).select_dtypes(include=['float64']).columns.tolist()

X[X_cat_features].isnull().sum()

sex       66
smoker    66
region    66
dtype: int64

In [225]:
cat_sim = SimpleImputer(strategy='most_frequent')

X[X_cat_features] = cat_sim.fit_transform(X[X_cat_features])

X[X_cat_features].isnull().sum()

sex       0
smoker    0
region    0
dtype: int64

In [226]:
X[X_num_features].isnull().sum()

age         66
bmi         66
children    66
dtype: int64

In [227]:
X[X_num_features] = X[X_num_features].fillna(X[X_num_features].mean())

X[X_num_features].isnull().sum()

age         0
bmi         0
children    0
dtype: int64

In [228]:
enc = OneHotEncoder(handle_unknown='ignore', sparse_output=False)

X_cat_encoded = enc.fit_transform(X[X_cat_features])
X_cat_encoded_df = pd.DataFrame(X_cat_encoded, columns=enc.get_feature_names_out(X_cat_features))
                                    
X = X[X_num_features].merge(X_cat_encoded_df, left_index=True, right_index=True)

X.head()

,age,bmi,children,sex_female,sex_male,smoker_no,smoker_yes,region_northeast,region_northwest,region_southeast,region_southwest
0,19.0,27.900,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
1,18.0,33.770,1.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0
2,28.0,33.000,3.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0
3,33.0,22.705,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0
4,32.0,28.880,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0


In [229]:
scr = StandardScaler()

X = scr.fit_transform(X)

print(X)

[[-1.48353486 -0.44782898 -0.74688488 ... -0.54914153 -0.66235163
   1.81341157]
 [-1.55648713  0.54022078  0.04022164 ... -0.54914153  1.50977209
  -0.55144679]
 [-0.82696444  0.41061289  1.61443467 ... -0.54914153  1.50977209
  -0.55144679]
 ...
 [-1.55648713  1.05865235 -0.74688488 ... -0.54914153  1.50977209
  -0.55144679]
 [-1.33763032 -0.80130504 -0.74688488 ... -0.54914153 -0.66235163
   1.81341157]
 [ 1.58046044 -0.25089231 -0.74688488 ...  1.82102417 -0.66235163
  -0.55144679]]


In [230]:
lin = LinearRegression()

lin.fit(X, y)

LinearRegression()

In [231]:
mse_score = - cross_val_score(lin, X, y, cv=5, scoring='neg_mean_squared_error')
mean_mse = np.mean(mse_scores)
print(mean_mse)

r2_score = - cross_val_score(lin, X, y, cv=5, scoring='r2')
mean_r2 = np.mean(r2_scores)
print(mean_r2)

41901813.860416286
-0.6995324692177185


In [232]:
validation_data_path = 'validation_dataset.csv'
X_test = pd.read_csv(validation_data_path)

X_test.head()

,age,sex,bmi,children,smoker,region
0,18.0,female,24.090000,1.0,no,southeast
1,39.0,male,26.410000,0.0,yes,northeast
2,27.0,male,29.150000,0.0,yes,southeast
3,71.0,male,65.502135,13.0,yes,southeast
4,28.0,male,38.060000,0.0,no,southeast


In [233]:
X_test.select_dtypes(include=['object']).apply(lambda x: x.value_counts()).T.stack()

sex     female       25.0
        male         25.0
smoker  no           32.0
        yes          18.0
region  northeast    11.0
        northwest    16.0
        southeast    14.0
        southwest     9.0
dtype: float64

In [234]:
X_test_cat_features = test.select_dtypes(include=['object']).columns.tolist()
X_test_num_features = test.select_dtypes(include=['float64']).columns.tolist()

test_enc = OneHotEncoder(handle_unknown='ignore', sparse_output=False)

X_test_cat_encoded = test_enc.fit_transform(X_test[X_test_cat_features])
X_test_cat_encoded_df = pd.DataFrame(X_test_cat_encoded, columns=enc.get_feature_names_out(X_test_cat_features))
                                    
X_test = X_test[X_test_num_features].merge(X_test_cat_encoded_df, left_index=True, right_index=True)

X_test.head()

,age,bmi,children,sex_female,sex_male,smoker_no,smoker_yes,region_northeast,region_northwest,region_southeast,region_southwest
0,18.0,24.090000,1.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
1,39.0,26.410000,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0
2,27.0,29.150000,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0
3,71.0,65.502135,13.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0
4,28.0,38.060000,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0


In [235]:
validation_predictions = lin.predict(X_test)
validation_data = X_test

validation_data['predicted_charges'] = validation_predictions

In [236]:
validation_data.loc[validation_data['predicted_charges'] < 1000, 'predicted_charges'] = 1000

validation_data.head()

,age,bmi,children,sex_female,sex_male,smoker_no,smoker_yes,region_northeast,region_northwest,region_southeast,region_southwest,predicted_charges
0,18.0,24.090000,1.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1000.0
1,39.0,26.410000,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,1000.0
2,27.0,29.150000,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,1000.0
3,71.0,65.502135,13.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,1000.0
4,28.0,38.060000,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,1000.0
